<a href="https://colab.research.google.com/github/ssundar6087/Deep-Learning-Mini-Course/blob/main/Keras/DL_Minicourse_Keras_Day_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Deconstructing the Model Part 1
Today, we'll look at the model. Note that 90% of the code will be the same as the previous notebook. Our focus is to play with the model architecture and see if we can improve results.

# Image Classification Keras

In [ ]:
# Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

## Load Dataset - CIFAR-10
The dataset has 10 classes with 50k training images and 10k test images

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = x_train / 255.0, x_test / 255.0

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Show Some Images

In [ ]:
plt.figure(figsize=(8,8))
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    plt.xlabel(classes[y_train[i][0]])
plt.show()

## **YOUR EXERCISE HERE: Define the Model  👇** 
In today's exercise, you'll be creating a convolutional neural network. Previously, we flatten a 3 channel image (32 x 32 x 3) into a vector and passed that through linear layers. It's now time to spruce up the model and pass the image directly through convolutional layers.

**Convolutions?**
The example below is for 2D convolutions, but generally, you have as many dimensions as you have in the input stream (images, audio, time-series, point clouds, etc.). $B$ is the batch size and $C$ is the number of channels. $dim$ is the resolution of the input which in this case is 32. Finally, $k$ is the size of the convolutional kernel.

  - What is my input shape: $B \times C \times dim \times dim$ 

  - What is my output shape: $B \times C \times dim' \times dim'$

  - Here $dim' = \frac{(dim + 2 \times padding - k)}{stride} + 1$


**HINT:** DO NOT SEE UNTIL YOU'VE TRIED THIS YOURSELF https://www.tensorflow.org/tutorials/images/cnn


In [ ]:
def get_baby_thanos(input_shape, num_classes):
  inputs = keras.Input(shape=input_shape)
  # YOUR CODE HERE

  # END CODE
  x = layers.Flatten()(x)
  x = keras.layers.Dense(100, activation='relu')(x)
  x = keras.layers.Dense(40, activation='relu')(x)
  outputs = keras.layers.Dense(num_classes)(x)
  return keras.Model(inputs, outputs)    

In [ ]:
IN_FTRS = (32, 32, 3) # CIFAR-10 images are 32 x 32 and have 3 channels
net = get_baby_thanos(input_shape=IN_FTRS, num_classes=10)

In [ ]:
net.summary()

In [ ]:
print(train_images.shape, test_images.shape)

## Hyperparameters 
**Note:** Use the best values from the previous exercise

In [ ]:
LEARNING_RATE = 0.001
EPOCHS = 20
BATCH_SIZE = 64

## Define Optimizer & Loss Function
These two functions allow us to help baby thanos learn from his mistakes.

In [ ]:
criterion = keras.losses.SparseCategoricalCrossentropy(from_logits=True) # Loss Function
optimizer = keras.optimizers.SGD(learning_rate=LEARNING_RATE) # Optimizer 

## Train and Evaluate the Network
The training and validation loops call the same set of functions over and over. Keras packages them into a nice function called `fit()` which does a lot more :)

In [ ]:
net.compile(
    optimizer=optimizer,
    loss=criterion,
    metrics=["accuracy"],
)

In [ ]:
history = net.fit(train_images, 
                  y_train, 
                  batch_size=BATCH_SIZE,
                  epochs=EPOCHS, 
                  validation_data=(test_images, y_test),
                  )

## Plot the Loss and Accuracy of our Model

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.grid("on")
plt.legend()
plt.title("Loss vs Epochs");

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.grid("on")
plt.legend()
plt.title("Accuracy vs Epochs");